In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")


In [3]:
df.head(5)


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)


,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape


(57650, 4)

In [6]:
df.isnull().sum()


artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)


In [8]:
df.head(10)


,artist,song,text
0,Europe,Tower's Callin',"All set ready to go, but little does he know ..."
1,Chuck Berry,Merry Christmas Baby,"Merry Christmas, baby, you really did treat me..."
2,Carpenters,We've Only Just Begun,We've only just begun to live \r\nWhite lace ...
3,Beautiful South,Alone,Like the contents of your handbag \r\nYou don...
4,Vince Gill,If You Ever Have Forever In Mind,"You're climbing mountains, I'm on a hill \r\n..."
5,Smiths,Stretch Out And Wait,All the lies that you make up \r\nWhat's at t...
6,Kirsty Maccoll,Teenager In Love,He's my boyfriend but he treats me so mean and...
7,Status Quo,Are You Growing Tired Of My Love,"You never seem, seem to have the time to liste..."
8,Ufo,"Push, It's Love",I take you walking out tonight \r\nI'm lookin...
9,Zao,Parade Of Chaos,We march like fools in the parade of chaos. \...


In [9]:
df['text'][0]


"All set ready to go, but little does he know  \r\nHe ain't comin' back no more  \r\nAll set ready to fly, into that deep blue sky  \r\nLike so many times before  \r\n  \r\nNow the tower's callin', there's no reply  \r\nAnd there's nothin' they can do  \r\nNow the night is fallin' before they eyes  \r\nStill no one's comin' through, callin'  \r\n  \r\nSomewhere out in time, God knows what's on his mind  \r\nIs there a reason for his doubt  \r\nSomehow unaware just like he didn't care  \r\nThere must be ways of gettin' out  \r\n  \r\nNot the tower's callin', there's no reply\r\n\r\n"

In [10]:
df.shape


(5000, 3)

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [24]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [26]:
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [28]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [29]:
similarity[0]


array([1.        , 0.03161813, 0.04016811, ..., 0.00758868, 0.015589  ,
       0.01446261])

In [41]:
df[df['song'] == 'Alone'].index[0]


3

In [46]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [43]:
import pickle


In [44]:
pickle.dump(similarity,open('similarity.pkl','wb'))


In [45]:
pickle.dump(df,open('df.pkl','wb'))